In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mlxtend.feature_selection import ExhaustiveFeatureSelector



import pandas as pd 
import numpy as np
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


In [ ]:
def forward_backward_FeatureSelection(Features,numberOfFeatures,Forward='forward'):
    '''
    #read Feature file as CSV 
    data = pd.read_csv(Features)

    LastFeature=numberOfFeatures
    target=-1

    #spilit Features from Trarget Class Variable
    X=data.iloc[:,0:LastFeature]
    Y=data.iloc[:,target]


    X_train,X_test,Y_train,Y_test=train_test_split(X,Y)


    foraward_feature_selection=SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
                                                         k_features=(1,LastFeature),
                                                         forward=Forward,
                                                         floating=False,
                                                         scoring="accuracy").fit(X_train,Y_train)


    to_ret=np.asarray(foraward_feature_selection.k_feature_names_)
    return to_ret
    '''

    # Load the data from a CSV file
    data = pd.read_csv(Features)

    # Separate the target variable from the features
    X = data.drop(data.columns[numberOfFeatures], axis=1)
    y = data[data.columns[numberOfFeatures]]

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the SVM model
    model = SVC(kernel='linear', C=1)

    # Initialize the Forward Feature Selection algorithm
    fwd_feature_selector = SequentialFeatureSelector(model, n_features_to_select=numberOfFeatures-1, direction='forward')

    # Fit the Forward Feature Selection algorithm to the training data
    fwd_feature_selector.fit(X_train, y_train)

    # Get the selected feature subset
    selected_features = X_train.columns[fwd_feature_selector.get_support()]

    # Save the selected features to a CSV file
    X_train[selected_features].to_csv('selected_features.csv', index=False)

    # Get the accuracy score on the test set
    accuracy = model.fit(X_train[selected_features], y_train).score(X_test[selected_features], y_test)
    print(f'Accuracy: {accuracy}')




In [ ]:
forward_backward_FeatureSelection("wine.csv",13)

In [ ]:
def PCA_FeatureSelection(input_file,numberOfFeatures):
    # #read Feature file as CSV 
    # data = pd.read_csv(Features)

    # #normalization
    # scalar=StandardScaler()
    # scalar.fit(data)
    # scaled_data=scalar.transform(data)


    # LastFeature=numberOfFeatures


    # #PCA_FeatureSelection
    # pca=PCA(n_components=numberOfFeatures)
    # pca.fit(scaled_data)
    # x_pca=pca.transform(scaled_data)
    

    # return x_pca


    # Load data from CSV file
    data = pd.read_csv(input_file)

    # Separate the target variable from the features
    X = data.drop(data.columns[-1], axis=1)
    y = data[data.columns[-1]]

    # Perform PCA
    pca = PCA(n_components=numberOfFeatures)
    X_transformed = pca.fit_transform(X)

    # Save the transformed data to a new CSV file
    new_data = pd.DataFrame(X_transformed, columns=[f'PC{i}' for i in range(1, numberOfFeatures+1)])
    new_data.insert(loc=0, column='Target', value=y)
    new_data.to_csv('output_file', index=False)

    print(f"PCA performed successfully. Transformed data saved to 'output_file'.")



    

In [ ]:
PCA_FeatureSelection("selected_features.csv",2)

In [ ]:

     #read Feature file as CSV 
data = pd.read_csv("wine.csv")

LastFeature=4
target=-1

#spilit Features from Trarget Class Variable
X=data.iloc[:,0:LastFeature]
Y=data.iloc[:,target]


X_train,_,Y_train,_=train_test_split(X, Y, test_size=0.3,
                                                  random_state= 1)


efs = ExhaustiveFeatureSelector(RandomForestClassifier(n_jobs=-1),
                              min_features=1,
                              max_features=LastFeature,
                              scoring= "accuracy",
                              cv = 2,
                              n_jobs=-1).fit(X_train, Y_train)


efs.best_feature_names_
efs.best_score_
pd.DataFrame.from_dict(efs.get_metric_dict()).T
